<a href="https://colab.research.google.com/github/worldwidekatie/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/module4-Hyperparameter-Tuning/LS_DS_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


## **Load data**

In [ ]:
import pandas as pd

In [ ]:
#Load the Data
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


## **Clean Data**

In [ ]:
df = df.replace({"Yes": 1, "No": 0, "No phone service": -1, " ": 0})
df = df.drop(['customerID'], axis=1)
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,1,0,1,0,-1,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,Male,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,Male,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,Male,0,0,0,45,0,-1,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,1,1,24,1,1,DSL,1,0,1,1,1,1,One year,1,Mailed check,84.80,1990.5,0
7039,Female,0,1,1,72,1,1,Fiber optic,0,1,1,0,1,1,One year,1,Credit card (automatic),103.20,7362.9,0
7040,Female,0,1,1,11,0,-1,DSL,1,0,0,0,0,0,Month-to-month,1,Electronic check,29.60,346.45,0
7041,Male,1,1,0,4,1,1,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Mailed check,74.40,306.6,1


In [ ]:
# This was just me figuring out why my one-hot-encoding was yielding
# thousands of columns. I fixed it in the replace above.

nonnum=[]

for i in df['TotalCharges']:
  i = str(i)
  for x in i:
    if x not in ['1','2','3','4','5','6','7','8','9','0','.']:
      nonnum.append(i)
nonnum

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

In [ ]:
nums = []

for i in df['TotalCharges']:
  nums.append(float(i))

df['Total_Charges'] = pd.Series(nums)
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Total_Charges
0,Female,0,1,0,1,0,-1,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0,29.85
1,Male,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0,1889.50
2,Male,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1,108.15
3,Male,0,0,0,45,0,-1,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,1840.75
4,Female,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1,151.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,1,1,24,1,1,DSL,1,0,1,1,1,1,One year,1,Mailed check,84.80,1990.5,0,1990.50
7039,Female,0,1,1,72,1,1,Fiber optic,0,1,1,0,1,1,One year,1,Credit card (automatic),103.20,7362.9,0,7362.90
7040,Female,0,1,1,11,0,-1,DSL,1,0,0,0,0,0,Month-to-month,1,Electronic check,29.60,346.45,0,346.45
7041,Male,1,1,0,4,1,1,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Mailed check,74.40,306.6,1,306.60


In [ ]:
X = df[['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'Total_Charges']]

y = df['Churn']

In [ ]:
!pip install category_encoders==2.*

In [ ]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X = encoder.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
print(X.shape)
print(y.shape)

(7043, 39)
(7043,)


## **Baseline**
Create and fit a baseline Keras MLP model to the data.

In [51]:
import tensorflow as tf
from tensorflow.keras.activations import relu
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam

In [55]:
model = Sequential()
model.add(Dense(39, activation='relu', input_shape=[39]))
model.add(Dense(39, activation='relu'))
model.add(Dense(1))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x=X_train, 
        y=y_train, 
        epochs=5, 
        validation_data=(X_test, y_test))

Epoch 1/5
177/177 [==============================] - 0s 3ms/step - loss: 11.2001 - accuracy: 0.2655 - val_loss: 11.2124 - val_accuracy: 0.2647
Epoch 2/5
177/177 [==============================] - 0s 2ms/step - loss: 11.2001 - accuracy: 0.2655 - val_loss: 11.2124 - val_accuracy: 0.2647
Epoch 3/5
177/177 [==============================] - 0s 2ms/step - loss: 11.2001 - accuracy: 0.2655 - val_loss: 11.2124 - val_accuracy: 0.2647
Epoch 4/5
177/177 [==============================] - 0s 2ms/step - loss: 11.2001 - accuracy: 0.2655 - val_loss: 11.2124 - val_accuracy: 0.2647
Epoch 5/5
177/177 [==============================] - 0s 2ms/step - loss: 11.2001 - accuracy: 0.2655 - val_loss: 11.2124 - val_accuracy: 0.2647


## **Hyperparameter Tuning**
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters

In [50]:
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

In [56]:
attempt = 0
attempts = []

# Now everything I do no matter what is giving the same scores and I don't know why it was all fine and then I ruined everything.

I'm gonna be done now because everything I do makes everything worse.

In [57]:
attempt += 1

param_grid = {'lr': [.5],
              'batch_size': [32],
              'epochs': [2],
              'optim': [Adam, SGD],
              'loss': ['binary_crossentropy', 'mse']}


layers = [Dense(39, activation='relu', input_shape=[39]), 
           Dense(1, activation='sigmoid')]

def create_model(lr, optim, loss):
    # Create Optimizer
    optim = optim(lr=lr)
    # create model
    model = Sequential()
    for l in layers:
      model.add(l)
    # Compile model
    model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

def report():
  # Report Results
  print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
      print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

  a=0
  b=0
  z=0
  columns=['attempt', 'score', 'stdev', 
                                          'batch_size', 'epochs', 
                                          'learningrt', 'optimizer', 
                                          'loss', 'num_layers']
  for l in layers:
    a+=1
    b+=1
    columns.append(f"L{a}_activ")
    columns.append(f"L{b}units")

  # Add results to table
  for mean, stdev, param in zip(means, stds, params):
    x = []
    z+=1

    x.append(f"{attempt}.{z}")
    x.append(mean)
    x.append(stdev)
    x.append(param['batch_size'])
    x.append(param['epochs'])
    x.append(param['lr'])
    opt = str(param['optim']).split(".")
    x.append(opt[-1].replace(">", ""))
    x.append(param['loss'])
    x.append(len(layers))
    for l in layers:
      x.append(l.get_config()['activation'])
      x.append(l.get_config()['units'])
    attempts.append(x)

  
  models = pd.DataFrame(attempts, columns=columns).sort_values(by='score', ascending=False) 
  return models


report()

Best: 0.7346286535263061 using {'batch_size': 32, 'epochs': 2, 'loss': 'binary_crossentropy', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 32, 'epochs': 2, 'loss': 'binary_crossentropy', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 32, 'epochs': 2, 'loss': 'binary_crossentropy', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.gradient_descent.SGD'>}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 32, 'epochs': 2, 'loss': 'mse', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 32, 'epochs': 2, 'loss': 'mse', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.gradient_descent.SGD'>}


,attempt,score,stdev,batch_size,epochs,learningrt,optimizer,loss,num_layers,L1_activ,L1units,L2_activ,L2units
0,1.1,0.734629,0.004758,32,2,0.5,Adam',binary_crossentropy,2,relu,39,sigmoid,1
1,1.2,0.734629,0.004758,32,2,0.5,SGD',binary_crossentropy,2,relu,39,sigmoid,1
2,1.3,0.734629,0.004758,32,2,0.5,Adam',mse,2,relu,39,sigmoid,1
3,1.4,0.734629,0.004758,32,2,0.5,SGD',mse,2,relu,39,sigmoid,1


# This one didn't work it gives all the same scores :(

In [ ]:
def neuralnets(name, parameters, layers):
  a=0
  b=0
  z=0
  
  lengths = []
  columns=['model_name', 'attempt', 'score', 'stdev', 
                                          'batch_size', 'epochs', 
                                          'learningrt', 'optimizer', 
                                          'loss', 'num_layers']

  for l in layers:
    lengths.append(len(l))

  for l in layers:
    if len(l) == max(lengths):
      for i in l:
        a+=1
        b+=1
        columns.append(f"L{a}_activ")
        columns.append(f"L{b}units")

  
  for l in layers:

    def create_model(lr, optim, loss):
        # Create Optimizer
        optim = optim(lr=lr)
        # create model
        model = Sequential()
        for i in l:
          model.add(i)
        # Compile model
        model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
        return model
  


    # create model
    model = KerasClassifier(build_fn=create_model, verbose=0)
    param_grid = parameters

    # Create Grid Search
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
    grid_result = grid.fit(X, y)

    # Report Results
    print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print(f"Means: {mean}, Stdev: {stdev} with: {param}") 


    # Add results to table
    for mean, stdev, param in zip(means, stds, params):
      x = []
      z+=1
      x.append(name)
      x.append(f"{attempt}.{z}")
      x.append(mean)
      x.append(stdev)
      x.append(param['batch_size'])
      x.append(param['epochs'])
      x.append(param['lr'])
      opt = str(param['optim']).split(".")
      x.append(opt[-1].replace(">", ""))
      x.append(param['loss'])
      x.append(len(l))
      for i in l:
        x.append(i.get_config()['activation'])
        x.append(i.get_config()['units'])
      attempts.append(x)

  
  models = pd.DataFrame(attempts, columns=columns).sort_values(by='score', ascending=False) 
  return models



attempt += 1

param_grid = {'lr': [.5],
              'batch_size': [32],
              'epochs': [2],
              'optim': [Adam, SGD],
              'loss': ['binary_crossentropy']}

layers = [[Dense(39, activation='relu', input_shape=[39]), 
           Dense(1, activation='sigmoid')], 
          [Dense(39, activation='relu', input_shape=[39]), 
           Dense(39, activation='relu', input_shape=[39]), 
           Dense(1, activation='sigmoid')]]

neuralnets("Variety", param_grid, layers)

Best: 0.7346286535263061 using {'batch_size': 32, 'epochs': 2, 'loss': 'binary_crossentropy', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 32, 'epochs': 2, 'loss': 'binary_crossentropy', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 32, 'epochs': 2, 'loss': 'binary_crossentropy', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.gradient_descent.SGD'>}
Best: 0.7346286535263061 using {'batch_size': 32, 'epochs': 2, 'loss': 'binary_crossentropy', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 32, 'epochs': 2, 'loss': 'binary_crossentropy', 'lr': 0.5, 'optim': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>}
Means: 0.7346286535263061, Stdev: 0.004

,model_name,attempt,score,stdev,batch_size,epochs,learningrt,optimizer,loss,num_layers,L1_activ,L1units,L2_activ,L2units,L3_activ,L3units
0,Variety,1.1,0.734629,0.004758,32,2,0.5,Adam',binary_crossentropy,2,relu,39,sigmoid,1,None,NaN
1,Variety,1.2,0.734629,0.004758,32,2,0.5,SGD',binary_crossentropy,2,relu,39,sigmoid,1,None,NaN
2,Variety,1.1,0.734629,0.004758,32,2,0.5,Adam',binary_crossentropy,3,relu,39,relu,39,sigmoid,1.0
3,Variety,1.2,0.734629,0.004758,32,2,0.5,SGD',binary_crossentropy,3,relu,39,relu,39,sigmoid,1.0
4,Variety,2.1,0.734629,0.004758,32,2,0.5,Adam',binary_crossentropy,2,relu,39,sigmoid,1,None,NaN
5,Variety,2.2,0.734629,0.004758,32,2,0.5,SGD',binary_crossentropy,2,relu,39,sigmoid,1,None,NaN
6,Variety,2.3,0.734629,0.004758,32,2,0.5,Adam',binary_crossentropy,3,relu,39,relu,39,sigmoid,1.0
7,Variety,2.4,0.734629,0.004758,32,2,0.5,SGD',binary_crossentropy,3,relu,39,relu,39,sigmoid,1.0


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation? **don't have to do this**
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?